# The Anatomy of a mewithoutYou Setlist
Exploring mewithoutYou's 2018-2019 [Untitled] Tour

## Standard Analysis Setup

In [ ]:
# Standard imports to run concert_analytics

import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(""), "../../"))
sys.path.insert(0, PROJECT_ROOT)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from postgres import fetch_table,run_query

import logging

logging.basicConfig(level=logging.INFO)  # DEBUG < INFO < WARNING < ERROR < CRITICAL

pd.set_option("display.max_columns", None)  # Show all columns

from plotnine import theme, element_text, element_blank, element_rect, element_line

# def rrg_theme(base_family_axis="Courier New", base_family_title="Helvetica"):
#     return (
#         theme(
#             # Backgrounds
#             plot_background=element_rect(fill="#FAF3E0", color=None),
#             panel_background=element_rect(fill="#FAF3E0", color=None),
#             panel_border=element_blank(),

#             # Margins around the entire plot
#             plot_margin_top= .0015, 
#             plot_margin_right=.0015,
#             plot_margin_bottom=.0015, 
#             plot_margin_left=.0015, 

#             # Titles & subtitles
#             plot_title=element_text(
#                 family=base_family_title,
#                 size=20,
#                 weight='bold',
#                 color='#1c1c1c',
#                 ha='left',
#                 margin={'b': 10}  # spacing below title
#             ),
#             plot_subtitle=element_text(
#                 family=base_family_title,
#                 size=14,
#                 color='#777777',
#                 ha='left',
#                 margin={'b': 12}  # spacing below subtitle
#             ),
#             plot_caption=element_text(
#                 family=base_family_axis,
#                 size=10,
#                 color='#777777',
#                 ha='right',
#                 margin={'t': 10}  # spacing above caption
#             ),

#             # Axis
#             axis_title=element_text(
#                 family=base_family_axis,
#                 size=10,
#                 color='#777777',
#                 weight='normal',
#                 margin={'t': 5, 'r': 5}  # tweak based on axis
#             ),
#             axis_text=element_text(
#                 family=base_family_axis,
#                 size=10,
#                 color='#1c1c1c'
#             ),
#             axis_ticks=element_line(color="#777777"),
#             axis_ticks_minor=element_blank(),
#             panel_grid_major=element_blank(),
#             panel_grid_minor=element_blank(),

#             # Legend
#             legend_title=element_blank(),
#             legend_background=element_blank(),
#             legend_key=element_blank(),
#             legend_margin=margins(t=5, b=5),
#             legend_box_margin=margins(t=10)
#         )
#     )

album_colors = {
    '[Untitled]': '#264653',
    '[A→B] Life': '#2a9d8f',
    'Catch for Us the Foxes': '#e9c46a',
    'Brother, Sister': '#f4a261',
    'It’s All Crazy...': '#e76f51',
    'Ten Stories': '#8d99ae',
    'Pale Horses': '#6a4c93'
}

## Fetch our dataset

In [ ]:
# fetch setlist and spotify metadata

query="""
with track_link_cte as (
	select 
 		sim.event_set_song_id
 		,sim.track_id
 		,row_number() over (partition by sim.event_set_song_id order by similarity_score desc nulls last) as similarity_rank_post_filter
	 from 
	 	analytics_mart.mart_track_setlist_similarity_scores as sim 
	 	join analytics_mart.mart_all_tracks as all_tr
	 		on sim.track_id = all_tr.track_id
	 where true
	 	and all_tr.album_id not in ('4bGIhBIGsziCakPziupVmQ','39P5R1B5XF1dGhpojrukVA','06y12naBReESEe7u2OFG7j')
)
, track_link_filtered_cte as (
	select 
 		*
	 from 
	 	track_link_cte
	 where true
	 	and similarity_rank_post_filter = 1
)
select 
	msh.*
	,tr.*
from 
	analytics_mart.mart_setlist_history as msh
	left join track_link_filtered_cte as tl_cte
		on msh.event_set_song_id = tl_cte.event_set_song_id
	left join analytics_mart.mart_all_tracks as tr
		on tl_cte.track_id = tr.track_id
where true
    and msh.event_tour_id = 'dff91b2e201d2bfdb4f7a16b57c62194'
    and msh.event_date <= '2019-01-01'
"""

df = run_query(query)
df.to_csv('mewithoutYou_Untitled_Tour_Data.csv')

## How many songs per night?

In [ ]:
# songs_per_night = 
songs_per_night = (
    df.groupby('event_date')['event_set_song_id']
    .nunique()
    .reset_index(name='unique_song_count')
    .sort_values('event_date')
)

total_shows = songs_per_night['event_date'].nunique()

average_songs = int(songs_per_night['unique_song_count'].mean())

print(f'{total_shows} shows with {average_songs} songs')


from plotnine import ggplot, aes, geom_col, labs, theme, element_text, element_rect, scale_x_datetime
from mizani.breaks import date_breaks
from mizani.formatters import date_format

base_family_axis="Courier New"
base_family_title="Helvetica"

# Build the plot
g = (
    ggplot(songs_per_night, aes(x='event_date', y='unique_song_count')) +
    geom_col(fill='#33C27D') +
    scale_x_datetime(date_labels="%b %d", breaks=date_breaks(width="2 week")) +
    labs(
        title=f"mewithoutYou played {total_shows} shows throughout \ntheir tour, playing {average_songs} songs on the average night",
        subtitle="Unique songs played by show in the 2018 [Untitled] tour",
        y="Unique Songs Played",
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()
        #     family=base_family_title+'-Bold'
        #     ,size=30
        #     ,linespacing=1.25
        #     ,weight='bold' 
        #     ,color='#1c1c1c'
        #     ,ha='left'
        #     ,margin={'b': 10 } # spacing below title
        # )

        , plot_subtitle=element_blank()
        #     family=base_family_title
        #     ,size=20
        #     ,color='#777777'
        #     ,ha='left'
        #     ,margin={'b': 30}  # spacing below subtitle
        # )
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_y = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_x =element_blank()
        ,panel_grid_minor=element_blank()

        # # Legend
        # ,legend_title=element_blank()
        # ,legend_background=element_blank()
        # ,legend_key=element_blank()
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )
)

# g

g.save(
    filename="0001-mewithoutYou-Songs-per-show.svg",
    format="svg",
    width=16,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

## How Many Unique Songs were Played in Total?

In [ ]:
def preview_palette(color_dict):
    fig, ax = plt.subplots(figsize=(8, 1))
    for i, (label, color) in enumerate(color_dict.items()):
        ax.bar(i, 1, color=color)
        ax.text(i, -0.3, label, ha='center', fontsize=10, rotation=45)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, len(color_dict)-0.5)
    ax.set_title("Custom Album Color Palette")
    plt.show()

preview_palette(album_colors)

In [ ]:
# songs_per_night = 
songs_albums = (
    df.groupby(['track_name','album_name'])['event_set_song_id']
    .nunique()
    .reset_index(name="times_played")
    .sort_values('times_played')
    .reset_index(drop=True)
)

songs_albums['track_name'] = songs_albums['track_name'].apply(
    lambda x: x if len(x) <= 25 else x[:22] + "..."
)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs, guides,guide_legend
from plotnine import theme, element_text
import pandas as pd

# Step 1: Make sure song order is preserved
songs_albums_sorted = songs_albums.sort_values('times_played', ascending=True).copy()
songs_albums_sorted['track_name'] = pd.Categorical(
    songs_albums_sorted['track_name'],
    categories=songs_albums_sorted['track_name'],
    ordered=True
)


# Step 2: Define the color scale
album_colors = {
    "A To B Life": "#A7ECF5",
    "Brother, Sister": "#F986BA",
    "Catch For Us The Foxes": "#3C7DC4",
    "It's All Crazy! It's All False! It's All A Dream! It's Alright": "#D64848",
    "Pale Horses": "#F25C05",
    "Pale Horses: Appendix":"#A7ECF5",
    "Ten Stories": "#5D4E8C",
    "[Untitled]": "#33C27D",
    "[untitled] e.p.": "#F6D357"
}

# Step 3: Create the plot
plot = (
    ggplot(songs_albums_sorted, aes(x='track_name', y='times_played', fill='album_name')) +
    geom_col(show_legend=True) +
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    labs(
        title="Song Frequency on the mewithoutYou [Untitled] Tour",
        subtitle="Each bar shows how many nights a song appeared during the tour",
        x="Song",
        y="Number of Times Played",
        fill=None,
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(12,16)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,weight='ultralight'
            # ,margin={'t': 5, 'r': 5}  # tweak based on axis
        )
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="top"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=2,reverse=True))
)

# plot


plot.save(
    filename="0001-mewithoutYou-song-frequency-chart.svg",
    format="svg",
    width=12,      # inches
    height=16,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

In [ ]:
# unique songs per album

# Count how many unique songs from each album were played
unique_songs_by_album = (
    songs_albums.groupby('album_name')['track_name']
    .nunique()
    .reset_index(name='unique_song_count')
    .sort_values('unique_song_count', ascending=True)
)


# Create a long-form DataFrame where each album is a segment of the bar
stacked_bar_data = unique_songs_by_album.copy()
stacked_bar_data['x'] = ""  # single category for stacking

from pandas.api.types import CategoricalDtype

# Set album_name as a categorical with explicit order
album_order = stacked_bar_data['album_name'].tolist()
album_cat = CategoricalDtype(categories=album_order, ordered=True)
stacked_bar_data['album_name'] = stacked_bar_data['album_name'].astype(album_cat)

# Step 2: Define the color scale
album_colors = {
    "A To B Life": "#A7ECF5",
    "Brother, Sister": "#F986BA",
    "Catch For Us The Foxes": "#3C7DC4",
    "It's All Crazy! It's All False! It's All A Dream! It's Alright": "#D64848",
    "Pale Horses": "#F25C05",
    "Pale Horses: Appendix":"#A7ECF5",
    "Ten Stories": "#5D4E8C",
    "[Untitled]": "#33C27D",
    "[untitled] e.p.": "#F6D357"
}

# stacked_bar_data.head(5)

from plotnine import ggplot, aes, geom_col, coord_flip, scale_fill_manual, labs

stacked_plot = (
    ggplot(stacked_bar_data, aes(x='x', y='unique_song_count', fill='album_name')) +
    geom_col(width=0.5) +  # single stacked bar
    coord_flip() +
    scale_fill_manual(values=album_colors) +
    labs(
        title="Unique Songs Played per Album on the [Untitled] Tour",
        subtitle="Stacked bar showing how many different songs were performed from each album",
        x=None,
        y="Number of Unique Songs",
        caption="Source: setlist.fm sourced via Concert Analytics"
    ) +
    theme(
        # aspect_ratio = 9 / 16
        # , 
        figure_size=(16,9)
        
        # Backgrounds
        , plot_background=element_rect(fill="#FAF3E0", color=None)
        , panel_background=element_rect(fill="#FAF3E0", color=None)
        , panel_border=element_blank()

        #Margins
        ,plot_margin_top= .03
        ,plot_margin_bottom=.025
        ,plot_margin_right= 0.03
        ,plot_margin_left=0.03

        # Titles & subtitles
        ,plot_title=element_blank()

        , plot_subtitle=element_blank()
        
        , plot_caption=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            ,ha='right'
            ,margin={'t': 30}  # spacing above caption
        )

        # Axis
        ,axis_title=element_blank()
        
        ,axis_text=element_text(
            family=base_family_axis
            ,size=12
            ,color='#777777'
            # ,margin={'l':5, 't':20}
        )

        ,axis_title_y=element_blank()
        ,axis_title_x=element_blank()
        
        ,axis_ticks=element_line(
            color="#FAF3E0"
        )
        ,panel_grid_major_x = element_line(
            color="#777777"
            ,linetype="dashed"
            ,size=.5
        )

        ,axis_ticks_minor=element_blank()
        ,panel_grid_major_y =element_blank()
        ,panel_grid_minor=element_blank()

        # Legend
        ,legend_title=element_blank()
        ,legend_background=element_blank()
        ,legend_key=element_blank()
        ,legend_key_width=10
        , legend_position="bottom"
        , legend_margin=1
        , legend_direction="vertical"
        # ,legend_margin=margins(t=5, b=5)
        # ,legend_box_margin=margins(t=10)

    
    )  + 
    guides(fill=guide_legend(ncol=4,reverse=True))  # your custom theme with background/colors/fonts
)

stacked_plot.save(
    filename="0001-mewithoutYou-Songs-per-Album-chart.svg",
    format="svg",
    width=16,      # inches
    height=9,     # adjust to your chart’s shape
    dpi=300        # high-quality output
)

In [ ]:
## Most Commonly Played Songs

In [ ]:
## Openers and Closers

In [ ]:
## Rarest Songs

In [ ]:
## Album Representation per Show

In [ ]:
## How popularity moves within a set

In [ ]:
## City that got the worst or best set list

In [ ]:
## How popularity moved for the tour (avg. Popularity per show)